<a href="https://colab.research.google.com/github/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/main_sudoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this project a software is developed which recognizes a Sudoku puzzle and solves it automatically.

![Sudoku Puzzle](https://github.com/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/Aufgabe.jpg?raw=1)

# Group members
* Justine Bruns (Business Informatics)

* Dennis Kempf (Computing Science)



# Use Cases and Limitations

- User requires assistence solving a Sudoku puzzle
  - User has not made any custom entries yet
  - User has made custom entries
    - User has made digital/computer-generated entries (e. g. in Sudoku app)
    - User has made handwritten entries (pen and paper Sudoku)
      - **Limitation**: We do not differentiate between handwritten and computer-generated digits
- A Sudoku puzzle may have multiple solutions (non-proper Sudoku)
  - **Limitation**: We only work with the first solution provided by solver algorithm
- Sudokus are differently colored
  - **Limitation**: We focus on Sudokus with light background and dark foreground (border/digits) preferrably black-on-white
- Photos are taken from different perspectives
  - **Limitation**: Sudoku must not be angled more than 45 degrees respective to the camera's view (also no horizontally/vertically flipped images)
- Sudokus with more than 9 and less than 9 digits/rows/columns exist (e. g. 8x8 or 7x7)
  - **Limitation**: We only consider 9x9 Sudokus but keep the possibility of different types of Sudokus during development in mind


# Implementation

Our implemenation of the application is split across multiple notebooks that work together like a pipeline.
Each notebook starts with a chapter "*Preamble*" which installs necessary packages and fetches output from previous stages.
At the end of each notebook there is a chapter "*Export*" which summarizes the respective results and uploads them to Google Drive for the following stages to continue working on.

The following graphic illustrates the basic concept of our processing pipeline:

![Illustration of our processing pipeline](https://github.com/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/documentation/pipeline.png?raw=true)

In the following each stage of our pipeline will be summarized and a link to the corresponding notebook will be provided where the exact steps are explained in depth.

## 0. Acquisition

Before we can actually start our pipeline we need data to feed it.
This data consists of input images taken using cameras and labels describing the image contents, e. g. what digit each cell contains.

[Link to the notebook on GitHub](https://github.com/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/main_acquisition.ipynb)

[![Open in Colab](https://raw.githubusercontent.com/uol-mediaprocessing/group-projects-sudoku-solver/master/documentation/open_in_colab.png)](https://colab.research.google.com/github/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/main_acquisition.ipynb)

## 1. Detection

The "real" first step is to detect the edges/corners of the Sudoku puzzle within the input image.
Detected contours that are most certainly incorrect will be filtered out automatically.

![Detection Visualization](https://github.com/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/documentation/detection.png?raw=true)

[Link to the notebook on GitHub](https://github.com/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/main_detection.ipynb)

[![Open in Colab](https://raw.githubusercontent.com/uol-mediaprocessing/group-projects-sudoku-solver/master/documentation/open_in_colab.png)](https://colab.research.google.com/github/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/main_detection.ipynb)

## 2. Transformation

With this knowledge we can compute a transformed view of the input image that looks like it was shot straigth from above.
We ask the developer to filter out all images where the transformation did not succeed as expected. 

![Transformation Visualization](https://github.com/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/documentation/transformation.png?raw=true)

[Link to the notebook on GitHub](https://github.com/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/main_transformation.ipynb)

[![Open in Colab](https://raw.githubusercontent.com/uol-mediaprocessing/group-projects-sudoku-solver/master/documentation/open_in_colab.png)](https://colab.research.google.com/github/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/main_transformation.ipynb)

## 3. Extraction

We now divide the image into 9*9 squares.
Because of the previous transformation each square should now contain exactly one cell of the Sudoku puzzle. 

![Extraction Visualization](https://github.com/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/documentation/extraction.png?raw=true)

[Link to the notebook on GitHub](https://github.com/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/main_extraction.ipynb)

[![Open in Colab](https://raw.githubusercontent.com/uol-mediaprocessing/group-projects-sudoku-solver/master/documentation/open_in_colab.png)](https://colab.research.google.com/github/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/main_extraction.ipynb)

## 4. Recognition

Using CNNs we classify each extracted cell as containing either a number from 1 to 9 or being empty (=0).

![Recognition Visualization](https://github.com/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/documentation/recognition.png?raw=true)

[Link to the notebook on GitHub](https://github.com/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/main_recognition.ipynb)

[![Open in Colab](https://raw.githubusercontent.com/uol-mediaprocessing/group-projects-sudoku-solver/master/documentation/open_in_colab.png)](https://colab.research.google.com/github/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/main_recognition.ipynb)

## 5. Solving

The CNN predictions are then fed into a Sudoku solving algorithm to find a solution for the empty cells.
Puzzles we cannot find a solution for will be discarded.

![Solving Visualization](https://github.com/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/documentation/solving.png?raw=true)

[Link to the notebook on GitHub](https://github.com/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/main_solving.ipynb)

[![Open in Colab](https://raw.githubusercontent.com/uol-mediaprocessing/group-projects-sudoku-solver/master/documentation/open_in_colab.png)](https://colab.research.google.com/github/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/main_solving.ipynb)

## 6. AR

The solver's solution is then projected on top of the input image in an AR manner.

![AR Visualization](https://github.com/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/documentation/ar.png?raw=true)

[Link to the notebook on GitHub](https://github.com/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/main_ar.ipynb)

[![Open in Colab](https://raw.githubusercontent.com/uol-mediaprocessing/group-projects-sudoku-solver/master/documentation/open_in_colab.png)](https://colab.research.google.com/github/uol-mediaprocessing/group-projects-sudoku-solver/blob/master/main_ar.ipynb)



